In [2]:
import numpy as np
import pandas as pd
import sympy
from sympy import *
from operator import mul
from functools import reduce
from mpmath import fac
import functools
from sympy import init_printing
from IPython.display import display_latex
init_printing()